In [2]:
import pandas as pd
import sqlite3
from datetime import datetime

# Load Excel file
df = pd.read_excel("/Users/marclambertes/Downloads/Wyscout database/Croatia 2024-2025.xlsx", sheet_name="Search results (319)")

# Connect to SQLite database (or create it)
conn = sqlite3.connect("scouting.db")
cursor = conn.cursor()

# Create tables
cursor.executescript("""
DROP TABLE IF EXISTS players;
DROP TABLE IF EXISTS teams;
DROP TABLE IF EXISTS contracts;
DROP TABLE IF EXISTS stats;

CREATE TABLE teams (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT UNIQUE
);

CREATE TABLE players (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    age INTEGER,
    position TEXT,
    team_id INTEGER,
    FOREIGN KEY (team_id) REFERENCES teams(id)
);

CREATE TABLE contracts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER,
    market_value INTEGER,
    contract_expires TEXT,
    FOREIGN KEY (player_id) REFERENCES players(id)
);

CREATE TABLE stats (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    player_id INTEGER,
    matches_played INTEGER,
    minutes_played INTEGER,
    goals INTEGER,
    assists INTEGER,
    FOREIGN KEY (player_id) REFERENCES players(id)
);
""")

# Helper function to get or insert team
team_cache = {}
def get_team_id(team_name):
    if team_name in team_cache:
        return team_cache[team_name]
    cursor.execute("INSERT OR IGNORE INTO teams (name) VALUES (?)", (team_name,))
    conn.commit()
    cursor.execute("SELECT id FROM teams WHERE name = ?", (team_name,))
    team_id = cursor.fetchone()[0]
    team_cache[team_name] = team_id
    return team_id

# Insert data
for _, row in df.iterrows():
    name = row["Player"]
    age = int(row["Age"]) if pd.notnull(row["Age"]) else None
    position = row["Position"]
    team_name = row["Team"]
    team_id = get_team_id(team_name)

    cursor.execute("INSERT INTO players (name, age, position, team_id) VALUES (?, ?, ?, ?)",
                   (name, age, position, team_id))
    player_id = cursor.lastrowid

    market_value = int(row["Market value"]) if pd.notnull(row["Market value"]) else None
    contract_expires = row["Contract expires"]
    if pd.notnull(contract_expires):
        try:
            contract_expires = pd.to_datetime(contract_expires).date().isoformat()
        except:
            contract_expires = None

    cursor.execute("INSERT INTO contracts (player_id, market_value, contract_expires) VALUES (?, ?, ?)",
                   (player_id, market_value, contract_expires))

    matches_played = int(row["Matches played"]) if pd.notnull(row["Matches played"]) else None
    minutes_played = int(row["Minutes played"]) if pd.notnull(row["Minutes played"]) else None
    goals = int(row["Goals"]) if pd.notnull(row["Goals"]) else None
    assists = int(row["Assists"]) if "Assists" in row and pd.notnull(row["Assists"]) else None

    cursor.execute("INSERT INTO stats (player_id, matches_played, minutes_played, goals, assists) VALUES (?, ?, ?, ?, ?)",
                   (player_id, matches_played, minutes_played, goals, assists))

conn.commit()
conn.close()
print("✅ Data loaded into scouting.db")


✅ Data loaded into scouting.db


In [1]:
import sqlite3
import pandas as pd
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk
from fpdf import FPDF

# Fetch data from SQLite
conn = sqlite3.connect("scouting.db")
query = """
SELECT 
    p.name AS Name, 
    t.name AS Team, 
    p.age AS Age, 
    p.position AS Position, 
    c.market_value AS 'Market Value', 
    c.contract_expires AS 'Contract Ends',
    s.goals AS Goals, 
    s.assists AS Assists, 
    s.minutes_played AS 'Minutes Played'
FROM players p
JOIN teams t ON p.team_id = t.id
JOIN contracts c ON p.id = c.player_id
JOIN stats s ON p.id = s.player_id
WHERE 
    p.age <= 30 AND 
    s.goals >= 5 AND 
    s.minutes_played > 1000 AND
    p.position LIKE '%CF%'
ORDER BY s.goals DESC
LIMIT 20;
"""
df = pd.read_sql_query(query, conn)
conn.close()

# Create GUI app
root = tk.Tk()
root.title("⚽ Scouting Dashboard")
root.geometry("1100x650")
root.configure(bg="#f0f2f5")

# Top frame for title and logo
top_frame = tk.Frame(root, bg="#f0f2f5")
top_frame.pack(fill="x", pady=10, padx=10)

# Title Label
title = tk.Label(top_frame, text="Top Scouting Targets (CFs Under 30)", font=("Helvetica", 16, "bold"), bg="#f0f2f5", fg="#333")
title.pack(side="left")

# Logo image (top right)
try:
    logo_img = Image.open("logo.png")
    logo_img = logo_img.resize((50, 50), Image.ANTIALIAS)
    logo = ImageTk.PhotoImage(logo_img)
    logo_label = tk.Label(top_frame, image=logo, bg="#f0f2f5")
    logo_label.image = logo
    logo_label.pack(side="right")
except Exception as e:
    print(f"Logo not loaded: {e}")

frame = tk.Frame(root, bg="#f0f2f5")
frame.pack(fill="both", expand=True, padx=10, pady=10)

# Scrollbars
scroll_y = tk.Scrollbar(frame, orient="vertical")
scroll_x = tk.Scrollbar(frame, orient="horizontal")

# Treeview (table)
tree = ttk.Treeview(frame, columns=list(df.columns), show="headings",
                    yscrollcommand=scroll_y.set, xscrollcommand=scroll_x.set, style="Custom.Treeview")
scroll_y.config(command=tree.yview)
scroll_y.pack(side="right", fill="y")
scroll_x.config(command=tree.xview)
scroll_x.pack(side="bottom", fill="x")

tree.pack(fill="both", expand=True)

# Apply styling
style = ttk.Style()
style.theme_use("default")
style.configure("Custom.Treeview",
                background="#ffffff",
                foreground="#333333",
                rowheight=30,
                fieldbackground="#ffffff",
                font=("Segoe UI", 10))
style.configure("Custom.Treeview.Heading",
                background="#004080",
                foreground="white",
                font=("Segoe UI", 10, "bold"))
style.map("Custom.Treeview",
          background=[("selected", "#cce5ff")],
          foreground=[("selected", "black")])

# Add column headers
for col in df.columns:
    tree.heading(col, text=col)
    tree.column(col, anchor="center", width=140)

# Add data rows
for _, row in df.iterrows():
    tree.insert("", "end", values=list(row))

# Save buttons
button_frame = tk.Frame(root, bg="#f0f2f5")
button_frame.pack(pady=10)

def save_as_csv():
    file_path = filedialog.asksaveasfilename(defaultextension=".csv", filetypes=[("CSV files", "*.csv")])
    if file_path:
        df.to_csv(file_path, index=False)
        messagebox.showinfo("Saved", f"Data saved to {file_path}")

def save_as_excel():
    file_path = filedialog.asksaveasfilename(defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx")])
    if file_path:
        df.to_excel(file_path, index=False)
        messagebox.showinfo("Saved", f"Data saved to {file_path}")

def save_as_pdf():
    file_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")])
    if file_path:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=10)
        col_width = 190 / len(df.columns)
        row_height = 8
        for col in df.columns:
            pdf.cell(col_width, row_height, col, border=1)
        pdf.ln(row_height)
        for _, row in df.iterrows():
            for item in row:
                pdf.cell(col_width, row_height, str(item), border=1)
            pdf.ln(row_height)
        pdf.output(file_path)
        messagebox.showinfo("Saved", f"Data saved to {file_path}")

btn_csv = tk.Button(button_frame, text="💾 Save as CSV", command=save_as_csv, bg="#004080", fg="black", padx=10, pady=5)
btn_csv.pack(side="left", padx=10)

btn_excel = tk.Button(button_frame, text="📊 Save as Excel", command=save_as_excel, bg="#007300", fg="black", padx=10, pady=5)
btn_excel.pack(side="left", padx=10)

btn_pdf = tk.Button(button_frame, text="📄 Save as PDF", command=save_as_pdf, bg="#800000", fg="black", padx=10, pady=5)
btn_pdf.pack(side="left", padx=10)

root.mainloop()

Logo not loaded: [Errno 2] No such file or directory: 'logo.png'


2025-04-07 19:40:01.625 python[30397:605630] +[CATransaction synchronize] called within transaction
Exception in Tkinter callback
Traceback (most recent call last):
  File "/Users/marclambertes/anaconda3/lib/python3.10/tkinter/__init__.py", line 1921, in __call__
    return self.func(*args)
  File "/var/folders/68/mhmqcpdn52943pyql2n4wj440000gn/T/ipykernel_30397/3835569148.py", line 135, in save_as_pdf
    pdf.output(file_path)
  File "/Users/marclambertes/anaconda3/lib/python3.10/site-packages/fpdf/fpdf.py", line 1065, in output
    self.close()
  File "/Users/marclambertes/anaconda3/lib/python3.10/site-packages/fpdf/fpdf.py", line 246, in close
    self._enddoc()
  File "/Users/marclambertes/anaconda3/lib/python3.10/site-packages/fpdf/fpdf.py", line 1636, in _enddoc
    self._putpages()
  File "/Users/marclambertes/anaconda3/lib/python3.10/site-packages/fpdf/fpdf.py", line 1170, in _putpages
    p = self.pages[n].encode("latin1") if PY3K else self.pages[n]
UnicodeEncodeError: 'latin-

In [2]:
pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40702 sha256=8c7729ab850da7f398c54793201241573f6a96727865e239593049222b5aa429
  Stored in directory: /Users/marclambertes/Library/Caches/pip/wheels/bc/6c/e7/c239668efef980927985e629d58674079552a9de5ec3aebfd8
Successfully built fpdf
Note: you may need to restart the kernel to use updated packages.
